In [20]:
%run graphika_trending.ipynb
%run misc_functions.ipynb
%run connect_to_api.ipynb

In [16]:
new_trends = get_today_trends()

>> Enter which type of hit you would like to look up (hashtags,urls,media): media


In [21]:
get_top_x_trends(search_trends(new_trends,auto = True))

>> Enter term to search: 
...Searching results
>> Enter how many of the top hits you would like to view: 5
...Getting a summary of top 5 hits
...Done!


,hit_count,hit_share,tweet_count,tweet_share,map_count,node_count,node_share
hit_value,,,,,,,
www.youtube.com/watch?v=6SmKIRrAYn4,56,0.31,56,0.32,1,1,0.02
www.youtube.com/watch?v=8d5E2sr3Zjc,21,0.11,21,0.12,1,1,0.02
twitter.com/KingOfQueenz/status/1245216838125928448/video/1,14,0.08,13,0.07,2,13,0.25
twitter.com/HOT97/status/1245324455687184384/photo/1,14,0.08,13,0.07,2,13,0.25
twitter.com/NickCannon/status/1245187190495166464/video/1,14,0.08,14,0.08,2,14,0.27


highest participation cluster lightup